In [13]:
pip install langchain faiss-cpu openai PyPDF2 pandas gdown langchain_openai langchain_community

INFO: pip is looking at multiple versions of requests[socks] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
zip_path = "/content/drive/MyDrive/nlp /dataset 4.zip"
extract_folder = "./cuad_dataset"

from zipfile import ZipFile
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Check extracted files
!ls "./cuad_dataset"


CUAD_v1


In [15]:
from PyPDF2 import PdfReader
import pandas as pd
import os

extract_folder = "./cuad_dataset"

#  Check subfolders
print("Top-level folders/files:", os.listdir(extract_folder))

pdf_dir = os.path.join(extract_folder, "CUAD_v1")

#  Check files inside pdf_dir
print("Files inside pdf_dir:", os.listdir(pdf_dir)[:10])

#  Load PDFs
def load_pdfs(pdf_folder):
    docs = []
    for root, _, files in os.walk(pdf_folder):  # walk recursively to find PDFs
        for file in files:
            if file.endswith(".pdf"):
                path = os.path.join(root, file)
                reader = PdfReader(path)
                text = ""
                for page in reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                if text.strip():  # only append if text is not empty
                    docs.append({"filename": file, "text": text})
    return pd.DataFrame(docs)

df_docs = load_pdfs(pdf_dir)
print("Number of documents loaded:", len(df_docs))
print("Columns:", df_docs.columns)
print(df_docs.head())


Top-level folders/files: ['CUAD_v1']
Files inside pdf_dir: ['full_contract_pdf', 'full_contract_txt', 'label_group_xlsx', 'CUAD_v1_README.txt', 'CUAD_v1.json', 'master_clauses.csv']
Number of documents loaded: 199
Columns: Index(['filename', 'text'], dtype='object')
                                            filename  \
0  NETGEAR,INC_04_21_2003-EX-10.16- AMENDMENT #2 ...   
1  NETGEAR,INC_04_21_2003-EX-10.16-DISTRIBUTOR AG...   
2  NETGEAR,INC_04_21_2003-EX-10.16-AMENDMENT TO T...   
3  FEDERATEDGOVERNMENTINCOMESECURITIESINC_04_28_2...   
4  FEDERATEDGOVERNMENTINCOMESECURITIESINC_04_28_2...   

                                                text  
0  This Amendment is entered into effective Octob...  
1                                                ...  
2  term under the Agreement will not be effective...  
3  LIMITED POWER OF ATTORNEY\nKNOW ALL MEN BY THE...  
4  Exhibit 28 (h) (1) (a) under Form N‐1A\nExhibi...  


In [5]:
from getpass import getpass
import os

api_key = getpass("Enter your OpenAI API Key: ")  # secure input
os.environ["OPENAI_API_KEY"] = api_key


Enter your OpenAI API Key: ··········


In [26]:
from getpass import getpass
import os

api_key = getpass("Enter your OpenAI API Key: ")
api_key = api_key.strip()
os.environ["OPENAI_API_KEY"] = api_key

print("API key set successfully.")


Enter your OpenAI API Key: ··········
API key set successfully.


In [27]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
print("Embedding model initialized successfully.")


Embedding model initialized successfully.


In [31]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")


In [33]:
!pip install sentence-transformers


In [35]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Initialize local embeddings (free)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Prepare texts and metadata
texts = df_docs["text"].tolist()
metadatas = [{"filename": f} for f in df_docs["filename"]]

# Create FAISS vector store
vector_store = FAISS.from_texts(texts, embedding_model, metadatas=metadatas)

# Save locally
vector_store.save_local("faiss_policy_index")
print("FAISS vector store created with local embeddings (free).")

/tmp/ipython-input-1532557830.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS vector store created with local embeddings (free).


In [43]:
import json

rules = [
    {"rule_id": 1, "description": "The contract must include confidentiality obligations."},
    {"rule_id": 2, "description": "The agreement must define termination conditions clearly."},
    {"rule_id": 3, "description": "There must be a clause specifying governing law."},
    {"rule_id": 4, "description": "The contract must include data protection requirements."},
    {"rule_id": 5, "description": "The agreement must identify all parties involved."},
    {"rule_id": 6, "description": "The contract must specify liability limitations."},
    {"rule_id": 7, "description": "The agreement must include an indemnification clause."},
    {"rule_id": 8, "description": "The contract must clearly define intellectual property rights."},
    {"rule_id": 9, "description": "The agreement must outline dispute resolution methods."},
    {"rule_id": 10, "description": "The contract must include a non-compete provision if applicable."},
    {"rule_id": 11, "description": "The agreement must state payment terms and obligations."},
    {"rule_id": 12, "description": "The contract must specify service-level expectations."},
    {"rule_id": 13, "description": "The agreement must outline audit rights for compliance verification."},
    {"rule_id": 14, "description": "The contract must include warranties and disclaimers."},
    {"rule_id": 15, "description": "The agreement must include amendment or modification conditions."}
]

with open("rules.json", "w") as f:
    json.dump(rules, f, indent=4)

print("rules.json created successfully.")


rules.json created successfully.


In [44]:
# Create retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

results = []

for rule in rules:
    rule_desc = rule["description"]
    docs = retriever.invoke(rule_desc)

    evidence = "\n---\n".join([doc.page_content for doc in docs])

    if evidence.strip():
        compliance_status = "Compliant (evidence found)"
    else:
        compliance_status = "Non-Compliant (no evidence)"

    results.append({
        "rule_id": rule["rule_id"],
        "rule": rule_desc,
        "status": compliance_status,
        "evidence": evidence
    })

import pandas as pd
compliance_df = pd.DataFrame(results)
compliance_df.to_csv("compliance_results.csv", index=False)
print(compliance_df.head())

   rule_id                                               rule  \
0        1  The contract must include confidentiality obli...   
1        2  The agreement must define termination conditio...   
2        3   There must be a clause specifying governing law.   
3        4  The contract must include data protection requ...   
4        5  The agreement must identify all parties involved.   

                       status  \
0  Compliant (evidence found)   
1  Compliant (evidence found)   
2  Compliant (evidence found)   
3  Compliant (evidence found)   
4  Compliant (evidence found)   

                                            evidence  
0  EXHIBIT 10.1\n                          BUSINE...  
1   \nEXHIBIT B\n \nMUTUAL TERMINATION AGREEMENT\...  
2                                           21\n ...  
3   \nExhibit 99.1\n \nCONFIDENTIAL  TREATMENT RE...  
4  EXHIBIT 10.1\n                          BUSINE...  


In [41]:
comparison_table = compliance_df[["rule_id", "rule", "status", "evidence"]]
comparison_table.to_csv("compliance_comparison.csv", index=False)
print(comparison_table)


    rule_id                                               rule  \
0         1                All employees must have unique IDs.   
1         2  Data encryption must be applied to sensitive f...   
2         3  All contracts must be reviewed by the legal team.   
3         4  Sensitive data must not be shared outside the ...   
4         5     Passwords must be at least 12 characters long.   
5         6   Access control must be enforced for all systems.   
6         7     All HR policies must be up-to-date and signed.   
7         8       Security audits must be performed quarterly.   
8         9          All contracts must mention governing law.   
9        10  Employees must complete mandatory compliance t...   
10       11        Data retention policies must be documented.   
11       12               All software licenses must be valid.   
12       13              Privacy policy must comply with GDPR.   
13       14   Incident response plan must exist and be tested.   
14       1

In [48]:
import json

with open("rules.json", "r") as f:
    rules = json.load(f)

# Convert rules into a dictionary using rule_id as key
rules_dict = {r["rule_id"]: r["description"] for r in rules}

print("Loaded", len(rules_dict), "rules.")


Loaded 15 rules.


In [51]:
def check_compliance(rule_id, retriever):
    if rule_id not in rules_dict:
        return "Invalid Rule ID."

    rule_text = rules_dict[rule_id]

    try:
        docs = retriever.invoke(rule_text)   # << FIXED LINE
    except Exception as e:
        return f"Retriever error: {e}"

    if not docs:
        return "No relevant context found for this rule."

    # combine chunks
    context = "\n\n".join([d.page_content[:800] for d in docs])

    # simple check (LLM-free)
    if any(word.lower() in context.lower() for word in rule_text.split()[:3]):
        status = "COMPLIANT"
    else:
        status = "NOT FOUND (POSSIBLY NON-COMPLIANT)"

    return f"Rule: {rule_text}\n\nStatus: {status}\n\nContext:\n{context[:1500]}"


In [52]:
import gradio as gr

rule_choices = [(f"Rule {rid}: {txt}", rid) for rid, txt in rules_dict.items()]

def ui_check(rule_id):
    return check_compliance(rule_id, retriever)

with gr.Blocks() as demo:
    gr.Markdown("## Policy Compliance Checker (RAG System)")

    rule = gr.Dropdown(
        choices=[c[0] for c in rule_choices],
        label="Select a rule",
        interactive=True
    )

    def map_rule(text):
        for name, rid in rule_choices:
            if text == name:
                return rid
        return None

    btn = gr.Button("Check Compliance")
    output = gr.Textbox(lines=12, label="Result")

    btn.click(
        fn=lambda t: ui_check(map_rule(t)),
        inputs=rule,
        outputs=output
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ef96eb9eedcddc19a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
